In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Set the device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the model and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

torch.save(model.state_dict(), 'human_detection_model.pth')
print(f"Model saved")

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
[1,   200] loss: 2.286
[1,   400] loss: 2.187
[1,   600] loss: 2.045
[2,   200] loss: 1.927
[2,   400] loss: 1.865
[2,   600] loss: 1.806
[3,   200] loss: 1.705
[3,   400] loss: 1.663
[3,   600] loss: 1.649
[4,   200] loss: 1.592
[4,   400] loss: 1.547
[4,   600] loss: 1.543
[5,   200] loss: 1.505
[5,   400] loss: 1.490
[5,   600] loss: 1.467
[6,   200] loss: 1.436
[6,   400] loss: 1.424
[6,   600] loss: 1.414
[7,   200] loss: 1.394
[7,   400] loss: 1.400
[7,   600] loss: 1.354
[8,   200] loss: 1.347
[8,   400] loss: 1.348
[8,   600] loss: 1.336
[9,   200] loss: 1.322
[9,   400] loss: 1.306
[9,   600] loss: 1.276
[10,   200] loss: 1.267
[10,   400] loss: 1.257
[10,   600] loss: 1.229
Finished Training
Model saved


In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from pycocotools.coco import COCO

# Define hyperparameters
learning_rate = 0.001
num_epochs = 10
batch_size = 32

# Define transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load COCO dataset
train_dataset = torchvision.datasets.CocoDetection(root='D:/СOCO Dataset/train2017', annFile='D:/СOCO Dataset/annotations/instances_train2017.json', transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Define model architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.features = torchvision.models.resnet18(pretrained=True)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Initialize model and loss function
model = Net()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print training progress
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Save trained model
torch.save(model.state_dict(), 'human_detection_model.pth')


loading annotations into memory...
